<a href="https://colab.research.google.com/github/young-scientific-researcher/Neurodegeneration/blob/main/segmentation_proj_new_preprocess_dapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install "opencv-python-headless<4.3"
# !pip install cellpose
import numpy as np
import time, os, sys
import cv2
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import PIL
import torch
from skimage.io import imread, imshow
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.filters import gaussian, sobel
from skimage.measure import label
from skimage.segmentation import watershed
from skimage.morphology import binary_opening
from skimage.feature import canny
from skimage.morphology import white_tophat, disk
from skimage.restoration import rolling_ball
import time
%matplotlib inline
mpl.rcParams['figure.dpi'] = 70
import cellpose
from urllib.parse import urlparse
from cellpose import models, core
from cellpose import utils
from cellpose import plot
from cellpose import io


# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
logger_setup()

(<Logger cellpose.io (WARNING)>, PosixPath('/root/.cellpose/run.log'))

In [ ]:
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

>>> GPU activated? 1


In [ ]:
!nvidia-smi

Wed Nov 15 11:26:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    27W /  70W |    105MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
z_stack_path_input = '/content/drive/MyDrive/cellpose/Alz/input/z_st_2/'

# read dapi

In [ ]:
paths_to_dapi = []

for filename in os.listdir(z_stack_path_input):
  f = os.path.join(z_stack_path_input, filename)

  if 'dapi.tif' in filename.lower():
    paths_to_dapi.append(f)
  else: continue
paths_to_dapi.sort(key=lambda x: int(os.path.basename(x).split('.')[0][11:14]))
dapi_imgs = [cv2.imread(f, 0) for f in paths_to_dapi]

In [ ]:
len(dapi_imgs)

176

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
cr_clahe = cv2.createCLAHE(10,(40, 40))
cr_clahe2 = cv2.createCLAHE(8,(15, 15))
filtered = []

start_time = time.time()
for i in dapi_imgs:
  cl_1 = cr_clahe.apply(i)
  blured = cv2.bilateralFilter(cl_1, 10, 20, 20)
  cl2 = cr_clahe2.apply(blured)
  closing = cv2.morphologyEx(cl2, cv2.MORPH_CLOSE, kernel, 5)
  closing = closing.astype(np.uint8)
  background_rolling = rolling_ball(closing, radius=30)
  closing_rolling = closing - background_rolling
  blur = cv2.bilateralFilter(closing_rolling,10,15,15)
  filtered.append(blur)

print("--- %s mins ---" % ((time.time() - start_time)/60))

--- 6.2460306207338965 mins ---


In [ ]:
filter_stack = np.stack(filtered, axis = 0)
filter_stack = np.moveaxis(filter_stack, 0, -1)
filter_stack.shape

(1024, 1024, 176)

In [ ]:
filter_stack_tiff = []
for i in range(filter_stack.shape[2]):
  img = filter_stack_tiff.append(PIL.Image.fromarray(filter_stack[:,:,i]))
filter_stack_tiff[0].save('/content/drive/MyDrive/cellpose/Alz/input/new_filter/new_filter.tif', compression="tiff_deflate", save_all=True,
               append_images=filter_stack_tiff[1:])

# 3D segmentation of filtered stack

In [ ]:
start_time = time.time()

model_f = models.Cellpose(gpu = True, model_type='cyto', net_avg = False, device = torch.device('cuda'))
channels = [0,0]
diameter_dapi = 44

masks_f3d, flows_f3d, styles_f3d, diams_f3d = model_f.eval(filter_stack, invert = True, diameter = diameter_dapi, do_3D = True, channels=channels, progress = True)

print("--- %s mins ---" % ((time.time() - start_time)/60))

In [ ]:
io.masks_flows_to_seg(raw_stack, masks_3d, flows_3d, diams_3d, '/content/drive/MyDrive/cellpose/Alz/input/new_filter/new_filter.tif', channels)